In [4]:
import pandapower as pp
import pandapower.networks as ppn
import numpy as np
import pandas as pd

In [5]:
#if network == 'IEEE-14' then
net = ppn.case14()
#else if network == 'IEEE-30' then
#net = pandapower.networks.case30()
#else if network == 'IEEE-57' then
#net = pandapower.networks.case57()
#else if network == 'IEEE-118' then
#net = pandapower.networks.case118()

In [14]:
# if user selects data generation strategy 1
pp.rundcpp(net)
# state vector x_base
x_base = net.res_bus.va_degree.to_numpy()
# number of discrete time steps
T = 100 # user argument
# Dx ~ N(0,Cov_Mat)
mean = np.zeros(x_base.shape)
cov = np.eye(x_base.shape[0])
delta_x_mat = np.transpose(np.random.multivariate_normal(mean, cov, size=T))
x_base_mat = np.repeat(x_base.reshape((x_base.shape[0],-1)), T, axis=1)
x_t = x_base_mat + delta_x_mat

numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)



NameError: name 'x' is not defined

In [156]:
# P_from_node_i_to_node_j
# Pij = (1/bij)*(x[i]-x[j])
# H[line_id,i] = 1/bij
# H[line_id,j] = -1/bij
A_real = np.real(net._ppc['internal']['Bbus'].A)
H = np.zeros((net.line.shape[0], net.bus.shape[0]))
for i in range(0, net.line.shape[0]):
    H[i, net.line.from_bus.values[i]] = 1/A_real[net.line.from_bus.values[i],net.line.to_bus.values[i]]
    H[i, net.line.to_bus.values[i]] = -1/A_real[net.line.from_bus.values[i],net.line.to_bus.values[i]]

In [157]:
#z_t = H @ x_t + noise
mean = np.zeros(H.shape[0])
cov = np.eye(H.shape[0])/10
noise_mat = np.transpose(np.random.multivariate_normal(mean, cov, size=T))
z_t_mat = np.matmul(H, x_t_mat) + noise_mat
z_t_mat.shape

(15, 100)

In [158]:
# if user selects data generation strategy 2
net = pandapower.networks.case14()

# TODO: https://towardsdatascience.com/stochastic-processes-simulation-the-ornstein-uhlenbeck-process-e8bff820f3

pp.rundcpp(net)
x_t = net.res_bus.va_degree.to_numpy()
x_t_mat = x_t.reshape((x_t.shape[0],-1))
for t in range(1,T):
    mean = np.zeros(net.load.p_mw.shape)
    cov = np.eye(net.load.p_mw.shape[0])    
    delta_load = np.random.multivariate_normal(mean, cov)
    net.load.p_mw.add(pd.Series(delta_load)) 
    pp.rundcpp(net)
    x_t = net.res_bus.va_degree.to_numpy().reshape((x_t.shape[0],-1))
    x_t_mat = np.hstack((x_t_mat, x_t))

In [159]:
#z_t = H @ x_t + noise
mean = np.zeros(H.shape[0])
cov = np.eye(H.shape[0])/10
noise_mat = np.transpose(np.random.multivariate_normal(mean, cov, size=T))
z_t_mat = np.matmul(H, x_t_mat) + noise_mat
z_t_mat.shape

(15, 100)